<a href="https://colab.research.google.com/github/sipocz/ingatlan/blob/master/ingatlan_com.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [124]:
import requests
from bs4 import  BeautifulSoup
from google.colab import drive
import time
drive.mount('/content/drive',force_remount=True)



def formatphonenumber(stri):
    if stri[0]=="0":
        stri="3"+stri[1:]
    if len(stri)<10:
        stri="36"+stri    
    out="+"+stri[0:2]+"-("+stri[2:4]+")-"+stri[4:7]+"-"+stri[7:]
    
    return out

def clearphonenumber(stri):
    out=""
    
    for i in stri:
        if i.isnumeric():
            out=out+i
    return(formatphonenumber(out))

def clearengunit(stri):
    out=""
    
    for i in stri:
        if i in ["0","1","2","3","4","5","6","7","8","9"]:
            out=out+i
    return(out)
def ansicode(sti):
    outstr=""
    for i in sti:
        a=i
        if i=="á":
            a="a"
        if i=="Á":
            a="A"
        if i=="é":
            a="e"
        if i=="É":
            a="E"
        if i=="í":
            a="I"
        if i=="Í":
            a="I"
        if i=="ö":
            a="o"
        if i=="Ö":
            a="O"
        if i=="ő":
            a="o"
        if i=="Ő":
            a="O"
        if i=="ó":
            a="o"
        if i=="Ó":
            a="O"
        if i=="ü":
            a="u"
        if i=="Ü":
            a="u"
        if i=="ú":
            a="u"
        if i=="Ú":
            a="u"
        outstr=outstr+a
    return(outstr)

   
        


def queryallinfo(url):
 
    res=requests.get(url)
    #print (res.text)
    soup=BeautifulSoup(res.text,"html.parser")
    
    phonelist=soup.find_all("p", class_="phone lead")
    if len(phonelist)==2:
        phone=phonelist[1].text
    else:
        phone="+36-99-999-9999"
    
    
    #print(phonelist[1].text)
    agentlist=soup.find_all("div", class_="agentName")
    
    if len(agentlist)==1:
        agent=agentlist[0].text
    else:
        agent="NONE"
    
    #print(agentlist[0].text)
    hazterulet=soup.find_all("li", title="Alapterület")
    #print(terulet)
    
    if len(hazterulet)>0:
        hazterulet=clearengunit(hazterulet[0].div.strong.text)
    else:
        hazterulet=0
    telekterulet= soup.find_all("li", title="Telekterület")
    #print(terulet)
    if len(telekterulet)>0:
        telekterulet=clearengunit(telekterulet[0].div.strong.text)
    else:
        telekterulet=0
    out=(agent,clearphonenumber(phone),hazterulet,telekterulet)
    return(out)


def querycity(city,pmin=20,pmax=40):
    header={'User-Agent':"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.75 Safari/537.36"}
    #print("------------------------------------------------")
    #print("+    ",city.capitalize())
    hnev=ansicode(city).lower()

    prefix="https://ingatlan.com/szukites/"
    url=prefix+"elado+haz+"+hnev+"+"+str(pmin)+"-"+str(pmax)+"-mFt"
    #print(url)
    res=requests.get(url,headers=header)
    #print (res.text)
    soup=BeautifulSoup(res.text,"html.parser")
    #print(soup)
    cardcontainer=soup.find_all("a", class_="listing__thumbnail js-listing-active-area")
    #print(cardcontainer)
    urllista=[]
    pricelist=[]
    for cc in cardcontainer:
        #print(cc['href'])
        urlf=cc["href"]
        urllist=urlf.split("/")
        code=urllist[-1]
        #print(code)
        urllista.append(code)

        #pricecontainer=cc.find_all("div",class_="figFav")
        #pricelist.append(pricecontainer[0]["data-price"])    
    
    
    return(urllista)
 
    '''
    for i in enumerate(urllist):
        link=prefix+i[1]
        o=(queryallinfo(link))
        name=o[0]
        num=o[1]
        haz=o[2]
        telek=o[3]
        print(f"{city:>15};{i[0]:>4}; {name:>40}; {num:>18}; {haz:>10}; {telek:>10}; {pricelist[i[0]]:>10}; Ft ;  {link:<1000}",file=openfile)
        print(f"{city:>15};{i[0]:>4}; {name:>40}; {num:>18}; {haz:>10}; {telek:>10}; {pricelist[i[0]]:>10}; Ft ;  {link:<1000}")
        #
    print("----------------  END  ------------------------")
'''

        
        
def query_all_link(city,querymin=0,querymax=150,querydelta=5):
    allist=[]
    print(city+":",end="")
    prefix="https://ingatlan.com/"
    for i in range(querymin,querymax,querydelta):
        #print(i,i+querydelta)
        allist=allist+querycity(city,pmin=i,pmax=i+querydelta)
        print(".", end="")
    allist=list(set(allist))
    allist=[city,prefix]+allist
    print("")
    print(allist)
    return(allist)












Mounted at /content/drive


In [125]:
#hely=open('/content/drive/My Drive/Ingatlan/fokusz.txt','r')

hely=["Zsámbék","Szomor","Gyermely"]
query_all_link("Zsámbék")
all_link=[]
for lak in hely:
    #openfile=open('/content/drive/My Drive/Ingatlan/lista_kedveltek.txt','a')
    telepules=lak.strip()
    all_link.append(query_all_link(telepules))

    time.sleep(5)
  



Zsámbék:..............................
['Zsámbék', 'https://ingatlan.com/', '29879930', '31772187', '31534730', '31606573', '31755713', '31654710', '31500120', '31757691', '29921276', '31557011', '31717678', '31710558', '31744694', '30512746', '31581220', '31707220', '31748276', '31605257', '31770021', '31795979', '31719501', '30651166', '31618525', '31820255', '31397362', '31729694', '31586220', '30761335', '31790569', '30388453', '31556108', '31661739', '31813573', '31195129', '31834701', '31725153', '31811376', '31763764', '31802403', '31725519', '28965770', '31792430', '30359477', '31570139', '31634068', '31543073', '31761893', '30513289', '31422140', '31707216', '31553773', '31772229', '30940885', '31546994', '28317098', '20170679', '31829869', '31441595']
Zsámbék:..............................
['Zsámbék', 'https://ingatlan.com/', '29879930', '31772187', '31534730', '31606573', '31755713', '31654710', '31500120', '31757691', '29921276', '31557011', '31717678', '31710558', '3174469

https://ingatlan.com/szukites/elado+haz+zsambek+20-30-mFt
